# POC PUCOMEX - Atendimento Serpro

## Instalação de pacotes necessários

In [1]:
# Importações básicas - Descomentar as linhas abaixo:
#!pip install pandas
#!pip install tensorflow
#!pip install keras
#!pip install scikit-learn
#!pip install nltk
#!pip install ezodf
#!pip install lxml
#import nltk
#nltk.download('all')
#!pip install spacy
#!python -m spacy download pt_core_news_sm

## Importações e criação de funções

In [2]:
import ezodf
import pandas as pd
import sys
import unicodedata
import re
import time
import spacy
from unicodedata import normalize
from nltk import tokenize # Tokenização
from nltk.corpus import stopwords # Stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import RSLPStemmer
# Função para mostrar as planilhas do documento
def show_ods_sheets(filename):
    doc = ezodf.opendoc(filename)
    print("Spreadsheet contains %d sheet(s)." % len(doc.sheets))
    for sheet in doc.sheets:
        print("-"*40)
        print("   Sheet name : '%s'" % sheet.name)
        print("Size of Sheet : (rows=%d, cols=%d)" % (sheet.nrows(), sheet.ncols()) )
# Função para converter ODS em DataFrame do Pandas
def read_sheet_ods_as_panda_dataframe(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})

## Abrindo o arquivo como um DataFrame

In [3]:
# Exibindo informações sobre o arquivo ODS
FILENAME = '/home/03662232677/workspace-neon/PUCOMEX/SS_PUCOMEX_Jan_2018 a Fev_2019_V2.ods'
show_ods_sheets(FILENAME)

Spreadsheet contains 2 sheet(s).
----------------------------------------
   Sheet name : 'SS'
Size of Sheet : (rows=18172, cols=17)
----------------------------------------
   Sheet name : 'Solution'
Size of Sheet : (rows=158, cols=11)


In [4]:
# Lendo o arquivo ODS com o Pandas
df = read_sheet_ods_as_panda_dataframe(FILENAME)

## Inspecionando e trabalhando o DataFrame

In [5]:
# Mostrando os primeiros registros
df.head()

,ticketid,ownergroup,owner,description,statusdate,externalsystem,serpronivelaten,descricaodemanda,solution,description2,UG_Atendimento,UG_Serviço,relatedreckey,internalpriority,SS_solucaoparausuario,Script_solucaoparausuario,None
0,2017SS/0000940433,UNACDFPUCOMEX,72503360106,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",13/03/2018 12:16,EMAIL,3NIVEL,"Bom Dia Tenho dois processos o qual ja voo , n...",,REGRA DE NEGÓCIO,SUNAC,SUNMF,,9,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP,None
1,2017SS/0000973776,UNACDFPUCOMEX,72503360106,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",18/01/2018 23:18,INTERNET,3NIVEL,"Bom dia, Enviei uma nota para o Siscomex e seu...",2016SOL/00396862,ANÁLISE URC,SUNAC,SUNMF,,9,"\n\nPrezado(a) usuário(a),\nProblema resolvido...","Sr.(a) usuário(a), conforme contato telefônic...",None
2,2017SS/0000986377,CAGSBRPUCOMEX,20656866772,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",04/01/2018 08:13,EMAIL,2NIVEL,,2016SOL/00399811,SITUAÇÃO NÃO PREVISTA,SUPCD,SUNMF,,9,"Prezado usuario,segue a informação do nosso de...","Sr.(a) usuário(a),conforme informado através d...",None
3,2017SS/0001008950,UNACDFPUCOMEX,72503360106,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",10/01/2018 09:28,EMAIL,3NIVEL,"Srs., Fizemos várias tentativas para obter inf...",2017SOL/00003655,REGRA DE NEGÓCIO,SUNAC,SUNMF,,9,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend...",None
4,2017SS/0001017121,UNACDFPUCOMEX,72503360106,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",10/01/2018 09:31,INTERNET,3NIVEL,Nro da Solicitação: 2017SS/0001012491: Como so...,2017SOL/0000002832,ANÁLISE URC,SUNAC,SUNMF,,9,"Caro usuário,\n\nA DUE 17BR0000079935 já se en...","Sr(a). Usuário(a),É necessário, para análise d...",None


In [6]:
# Selecionando apenas as colunas desejadas
df = df[['description','ownergroup','descricaodemanda','solution','SS_solucaoparausuario','Script_solucaoparausuario']]

In [7]:
# Mudando o nome das colunas
df.columns = ['descricao','grupo','descricao_demanda','solucao','solucao_usuario','solucao_script']
df.head()

,descricao,grupo,descricao_demanda,solucao,solucao_usuario,solucao_script
0,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",UNACDFPUCOMEX,"Bom Dia Tenho dois processos o qual ja voo , n...",,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP
1,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",UNACDFPUCOMEX,"Bom dia, Enviei uma nota para o Siscomex e seu...",2016SOL/00396862,"\n\nPrezado(a) usuário(a),\nProblema resolvido...","Sr.(a) usuário(a), conforme contato telefônic..."
2,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",CAGSBRPUCOMEX,,2016SOL/00399811,"Prezado usuario,segue a informação do nosso de...","Sr.(a) usuário(a),conforme informado através d..."
3,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",UNACDFPUCOMEX,"Srs., Fizemos várias tentativas para obter inf...",2017SOL/00003655,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend..."
4,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",UNACDFPUCOMEX,Nro da Solicitação: 2017SS/0001012491: Como so...,2017SOL/0000002832,"Caro usuário,\n\nA DUE 17BR0000079935 já se en...","Sr(a). Usuário(a),É necessário, para análise d..."


In [8]:
# Exibindo estatísticas do pandas
df.describe()

,descricao,grupo,descricao_demanda,solucao,solucao_usuario,solucao_script
count,18161,18161,18161,18161,18161,18161
unique,66,34,17059,174,9208,149
top,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",ATGSMG,,,,#N/DISP
freq,4216,4654,474,6564,2118,6737


In [9]:
# Removendo valores vazios em descricao_demanda
df = df.dropna(subset=['descricao_demanda']) 
df = df[df.descricao_demanda != '']
df.describe()

,descricao,grupo,descricao_demanda,solucao,solucao_usuario,solucao_script
count,17687,17687,17687,17687,17687,17687
unique,66,33,17058,174,8918,149
top,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",ATGSMG,URGENTE,,,#N/DISP
freq,4071,4536,14,6405,2067,6571


In [10]:
# removendo colunas indesejadas de grupo
df = df.dropna(subset=['grupo']) 
df = df[df.grupo != '']
grupos = df['grupo'].unique()
print(len(grupos))
grupos

33


array(['UNACDFPUCOMEX', 'GSDERJCCTR', 'CAGSBRPUCOMEX', 'ATGSBRTERC',
       'ATGSMG', 'CAGSBRNOVOEX', 'CAGSBRCOMEX', 'GSDESCPLATAFORMA',
       'GSGSSPCENTRODECOMANDO', 'GSCDSP', 'GSDERJDUEX', 'GSDESCGRIS',
       'GSDESCTALP', 'GSGSDFCENTRODECOMANDO', 'CAGSBRTRIBUT',
       'CAGSRSMDIC', 'GSDERJGATEWAY', 'ATGSSPRFB', 'ATGSPE', 'GSDERSOEA',
       'CACDBRPUCOMEX', 'CAGSBRDWRFB', 'GSDERJCOAD', 'GSDERJSUCE',
       'CAGSBRDRAWBACK', 'CACDBRCOMEX', 'GSCDSPASLGESTAOAT', 'ATGSSP',
       'ATCDMGRAIS', 'GSOPSPSEGI', 'GSOPSPARLDI', 'CACDBRTRIBUT',
       'ATGSBREMAIL'], dtype=object)

In [11]:
# Removendo quebra de linha da descricao
df['descricao'] = df['descricao'].replace(r'\n','', regex=True)
descricoes = df['descricao'].unique()
print(len(descricoes))
descricoes

66


array(['CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO',
       'CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARGA E TRÂNSITO,ANÁLISE URC',
       'CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,ANÁLISE URC',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO EXTERIOR,ANÁLISE URC',
       'CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARGA E TRÂNSITO,APLICAÇÃO',
       'CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO EXTERIOR,SITUAÇÃO NÃO PREVISTA',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO EXTERIOR,DÚVIDA OU INFORMAÇÕES',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO EXTERIOR,APLICAÇÃO,APLICAÇÃO VIA FORM-WEB',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO EXTERIOR,AMBIENTE,AMBIENTE VIA FORM-WEB',
       'CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,DÚVIDA OU INFORMAÇÕES',
       'CENTRAL DE 

In [12]:
# Extraindo o segundo e terceiro termo após a vírgula
import numpy as np
df['sistema'] =  (np.where(df['descricao'].str.contains(','),
                  df['descricao'].str.split(',').str[1],
                  df['descricao']))
df['classificacao'] =  (np.where(df['descricao'].str.contains(','),
                  df['descricao'].str.split(',').str[2],
                  df['descricao']))
# Reorganizando a ordem das colunas
df = df[['sistema','classificacao','grupo', 'descricao_demanda','descricao','solucao','solucao_usuario','solucao_script']]
# Removendo valores nulos das colunas criadas
df = df.dropna(subset=['sistema','classificacao'])
df = df[df.sistema != '']
df = df[df.classificacao != '']
df.describe()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
count,15610,15610,15610,15610,15610,15610,15610,15610
unique,14,13,29,15047,47,168,8298,143
top,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CACDBRPUCOMEX,URGENTE,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",,,#N/DISP
freq,6376,8078,4187,14,4071,4842,1636,4997


In [13]:
# Verificando quantos registros existem por sistema
df.groupby('sistema').count()

,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
sistema,,,,,,,
CA - CONFERÊNCIA ADUANEIRA EXPORTAÇÃO,60,60,60,60,60,60,60
CADASTRO DE INTERVENIENTES - RFB,1,1,1,1,1,1,1
CCT - CONTROLE DE CARGA E TRÂNSITO,3719,3719,3719,3719,3719,3719,3719
DA - DESPACHO ADUANEIRO,3,3,3,3,3,3,3
DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,6376,6376,6376,6376,6376,6376,6376
DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - MDIC,1,1,1,1,1,1,1
DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - RFB,2,2,2,2,2,2,2
GR – GERENCIAMENTO DE RISCO ALEATÓRIOS,1,1,1,1,1,1,1
OPERADOR ECONÔMICO AUTORIZADO (OEA) - RFB,1,1,1,1,1,1,1


In [14]:
# Escolhendo um sistema para a POC
SISTEMA_SELECIONADO = 'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO'
df = df.loc[df['sistema'] == SISTEMA_SELECIONADO]
# Mostrando quandos existem por classificacao
df.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
classificacao,,,,,,,
AMBIENTE,362,362,362,362,362,362,362
ANÁLISE URC,130,130,130,130,130,130,130
APLICAÇÃO,471,471,471,471,471,471,471
DÚVIDA OU INFORMAÇÕES,1258,1258,1258,1258,1258,1258,1258
HOMOLOGAÇÃO,1,1,1,1,1,1,1
REGRA DE NEGÓCIO,81,81,81,81,81,81,81
SITUAÇÃO DE CRISE,2,2,2,2,2,2,2
SITUAÇÃO NÃO PREVISTA,4071,4071,4071,4071,4071,4071,4071


In [15]:
# Removendo classificaoes com menos que X excemplos
MINIMO_REGISTROS = 50
counts = df['classificacao'].value_counts()
df = df[df['classificacao'].isin(counts[counts > MINIMO_REGISTROS].index)]
df.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
classificacao,,,,,,,
AMBIENTE,362,362,362,362,362,362,362
ANÁLISE URC,130,130,130,130,130,130,130
APLICAÇÃO,471,471,471,471,471,471,471
DÚVIDA OU INFORMAÇÕES,1258,1258,1258,1258,1258,1258,1258
REGRA DE NEGÓCIO,81,81,81,81,81,81,81
SITUAÇÃO NÃO PREVISTA,4071,4071,4071,4071,4071,4071,4071


In [16]:
df2 = df[df.classificacao == 'SITUAÇÃO NÃO PREVISTA']
df = df[df.classificacao != 'SITUAÇÃO NÃO PREVISTA']
df.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
classificacao,,,,,,,
AMBIENTE,362,362,362,362,362,362,362
ANÁLISE URC,130,130,130,130,130,130,130
APLICAÇÃO,471,471,471,471,471,471,471
DÚVIDA OU INFORMAÇÕES,1258,1258,1258,1258,1258,1258,1258
REGRA DE NEGÓCIO,81,81,81,81,81,81,81


In [17]:
df2.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
classificacao,,,,,,,
SITUAÇÃO NÃO PREVISTA,4071,4071,4071,4071,4071,4071,4071


In [18]:
classificacoes = df['classificacao'].unique()
print(len(classificacoes))
classificacoes

5


array(['REGRA DE NEGÓCIO', 'ANÁLISE URC', 'DÚVIDA OU INFORMAÇÕES',
       'AMBIENTE', 'APLICAÇÃO'], dtype=object)

In [19]:
df.head()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
0,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO,UNACDFPUCOMEX,"Bom Dia Tenho dois processos o qual ja voo , n...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP
3,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO,UNACDFPUCOMEX,"Srs., Fizemos várias tentativas para obter inf...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/00003655,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend..."
5,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,ANÁLISE URC,UNACDFPUCOMEX,"Recepcionamos a DUE 17BR0000257710, porem nao ...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",,"Caro Usuário,\n\nFavor informar a chave da not...",#N/DISP
9,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,ANÁLISE URC,UNACDFPUCOMEX,A cia aérea não consegue averbar a DUE 17BR000...,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/0000002832,"\nPrezado(a) Usuário(a),\n\nA DUE encontra-se ...","Sr(a). Usuário(a),É necessário, para análise d..."
11,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO,UNACDFPUCOMEX,GOSTARIA DE DETALHES DO MOTIVO DA NÃO AVERBAÇÃ...,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2016SOL/00397182,"Prezado Usuário,\n\nO manifestador precisa ret...","Prezado(a) usuário(a),conforme informado atrav..."


## Preparação dos dados

In [20]:
# Estrutura para armazenar pontuações
tbl = dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))
# Função para remover pontuações das sentenças
def remove_punctuation(text):
    return text.translate(tbl)

In [21]:
# Lista de palavras para inluir com regex
INCLUDE_WORDS_BY_REGEX = [r"^[A-Za-zçãàáâéêíóôõúüÂÃÁÀÉÊÍÓÔÕÚÜÇ'º°ª\.]*$",r"^[0-9\-/\\\.]*$"]
def include_words_by_regex(words):
    # Remove regex words
    words_aux = []
    for word in words:
        add = False
        for regex_exp in INCLUDE_WORDS_BY_REGEX:
            if re.compile(regex_exp).match(word):
                add = True
        if add:
            words_aux.append(word)
    return words_aux       
# Padroes regex para serem removidos do texto original
REMOVE_WORDS_BY_REGEX = [r"\S*@\S*\s?", # Email
    r"(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/[a-z0-9]*)*(\/)?", # Site
    r"([a-z0-9]+(\/)[a-z0-9]+)", # Protocolo
    r"(\(?\d{2,4}\)?)??[\-\s\.]?(\d{3,5})[\.\-]\d{4,5}", # Telefone
    r"\d{2}[\-\s\.]\d{4}", # Ramal
    r"([0-9]{2}[\.]?[0-9]{3}[\.]?[0-9]{3}[\/]?[0-9]{4}[-]?[0-9]{2})|([0-9]{3}[\.]?[0-9]{3}[\.]?[0-9]{3}[-]?[0-9]{2})", # CPF e CNPJ
    r"\d+" # Numeros
    #r"e-?mail:?" # texto email/e-mail
]
def remove_words_by_regex(text):
    for regex_exp in REMOVE_WORDS_BY_REGEX:
        text = re.sub(re.compile(regex_exp),"",text)
    return text
# Padroes regex para remoção do conteúdo a seguir
REMOVE_TEXT_AFTER_REGEX = [
        r"o acesso ao conte(u|ú)do desta mensagem",
        r"aviso legal",
        r"esta mensagem (e|é) destinada exclusivamente",
        r"legal advice",
        r"this message is exclusively destined"
]
def remove_text_after_regex(text):
    for regex_exp in REMOVE_TEXT_AFTER_REGEX:
        m = re.search(regex_exp, text)
        if m:
            text = text[:m.start()]
    return text

In [22]:
text = """"
srs., fizemos várias tentativas para obter informações em relação à adesão 
ao portal due, alguns exemplos: primeiramente acessamos o site: 
http://www.serpro.gov.br/menu/contato/cliente/ depois por telefone: 
0800-728-2323 e também nos contatos: 
sccd@serpro.gov.br<mailto:sccd@serpro.gov.br> 
http://comexresponde.comexbrasil.gov.br/ - protocolo: 30696 em uma das 
orientações recebidas (protocolo nº 2017ss/0000973710), nos informaram que 
o acesso seria através do mdic / comex, procedemos de acordo, porém não 
houve retorno. por favor, precisamos do contato urgente para alinhamento. 
favor contatar: fernanda cezar no telefone:3646-2256 / e-mail: 
fernanda.cezar@bradesco.com.br gratos. atenciosamente. banco bradesco s/a 
4861 / droc – departamento de relacionamento operacional com clientes setor 
apoio a projetos patrícia vidal tel: (11) 2646-3320 r: 31-3320 
classificação: interna “o acesso ao conteúdo desta mensagem está 
autorizado, exclusivamente, a colaboradores da organização bradesco. a 
reprodução a pessoas não enquadradas deve ser autorizada pelo gestor da 
informação, para identificá-lo contate o remetente". 
[cid:image164146.png@3690fe11.49984c89] aviso legal
...esta mensagem é destinada exclusivamente para a(s) pessoa(s) a quem é 
dirigida, podendo conter informação confidencial e/ou legalmente 
privilegiada. se você não for destinatário desta mensagem, desde já fica 
notificado de abster-se a divulgar, copiar, distribuir, examinar ou, de 
qualquer forma, utilizar a informação contida nesta mensagem, por ser 
ilegal. caso você tenha recebido esta mensagem por engano, pedimos que nos 
retorne este e-mail, promovendo, desde logo, a eliminação do seu conteúdo 
em sua base de dados, registros ou sistema de controle. fica desprovida de 
eficácia e validade a mensagem que contiver vínculos obrigacionais, 
expedida por quem não detenha poderes de representação. legal advice
...this message is exclusively destined for the people to whom it is 
directed, and it can bear private and/or legally exceptional information. 
if you are not addressee of this message, since now you are advised to not 
release, copy, distribute, check or, otherwise, use the information 
contained in this message, because it is illegal. if you received this 
message by mistake, we ask you to return this email, making possible, as 
soon as possible, the elimination of its contents of your database, 
registrations or controls system. the message that bears any mandatory 
links, issued by someone who has no representation powers, shall be null or 
void. [screenshot 1 saved as attachment image1641461512409665054.png by 
e-mail listener.]
"""
text = remove_words_by_regex(text.lower())
print(text)
text2 = remove_text_after_regex(text)
print(text2)

"
srs., fizemos várias tentativas para obter informações em relação à adesão 
ao portal due, alguns exemplos: primeiramente acessamos o site: 
 depois por telefone: 
 e também nos contatos: 

 - protocolo:  em uma das 
orientações recebidas (protocolo nº ), nos informaram que 
o acesso seria através do mdic / comex, procedemos de acordo, porém não 
houve retorno. por favor, precisamos do contato urgente para alinhamento. 
favor contatar: fernanda cezar no telefone: / e-mail: 
gratos. atenciosamente. banco bradesco  
 / droc – departamento de relacionamento operacional com clientes setor 
apoio a projetos patrícia vidal tel:  r:  
classificação: interna “o acesso ao conteúdo desta mensagem está 
autorizado, exclusivamente, a colaboradores da organização bradesco. a 
reprodução a pessoas não enquadradas deve ser autorizada pelo gestor da 
informação, para identificá-lo contate o remetente". 
aviso legal
...esta mensagem é destinada exclusivamente para a(s) pessoa(s) a quem é 
dirigida, p

In [23]:
spacy_pln = spacy.load('pt_core_news_sm')
words = [token.lemma_ for token in spacy_pln(text2)]
print (words)

['"', '\n', 'sr', '.', ',', 'fazer', 'vários', 'tentativo', 'parir', 'obter', 'informação', 'em', 'relação', 'à', 'adesão', '\n', 'o', 'o', 'portal', 'due', ',', 'algum', 'exemplo', ':', 'primeiramente', 'acessamos', 'o', 'site', ':', '\n ', 'depois', 'por', 'telefonar', ':', '\n ', 'e', 'também', 'o', 'contatos', ':', '\n\n ', '-', 'protocolo', ':', ' ', 'em', 'umar', 'das', '\n', 'orientação', 'recebido', '(', 'protocolo', 'nº', ')', ',', 'o', 'informar', 'que', '\n', 'o', 'acesso', 'seriar', 'através', 'do', 'mdic', '/', 'comex', ',', 'proceder', 'de', 'acordar', ',', 'porém', 'não', '\n', 'haver', 'retornar', '.', 'por', 'favor', ',', 'precisar', 'do', 'contato', 'urgente', 'parir', 'alinhamento', '.', '\n', 'favor', 'contatar', ':', 'fernanda', 'cezar', 'o', 'telefonar', ':', '/', 'e', '-', 'mail', ':', '\n', 'grato', '.', 'atenciosamente', '.', 'banco', 'bradesco', ' \n ', '/', 'droc', '–', 'departamento', 'de', 'relacionamento', 'operacional', 'com', 'cliente', 'setor', '\n', 'a

In [24]:
spacy_pln = spacy.load('pt_core_news_sm')
# Função para realizar o pré-processamento do texto livre
# Inicializa o stemmer
#stemmer = LancasterStemmer() 
stemmer = RSLPStemmer()
# Stop words para o Português
portuguese_stops = set(stopwords.words('portuguese'))
portuguese_stops.update('é')
REMOVER = ['attach','email']
LIMITE_TAMANHO_PALAVRA = 2
# Dicionátio que armazena as pontuações a serem removidas
def preprocessing(content):
    # Converte para minúsculo
    content = content.lower()
    # Remove elementos do texto via REGEX
    content = remove_words_by_regex(content)
    content = remove_text_after_regex(content)
    #Remover pontuação
    content = remove_punctuation(content)
    # Tokenização
    words = tokenize.word_tokenize(content, language='portuguese')
    #words = [token.lemma_ for token in spacy_pln(content)]
    # Remover acentuação
    words = [normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in words]
    # Remove stopwords
    words = [word for word in words if word.lower() not in portuguese_stops] 
    # Stemming de cada palavra
    words = [stemmer.stem(w) for w in words if w.strip()] 
    # Remove palavras menores que um limite
    words = [word for word in words if len(word) > LIMITE_TAMANHO_PALAVRA] 
    # Remove by regex
    words = include_words_by_regex(words)
    # Remove specific words
    words = [word for word in words if word not in REMOVER] 
    return ' '.join(words)

In [25]:
# Imprimindo alguns exemplos
for sample in df['descricao_demanda'].head(5):
    print('============================================================================')
    print(sample)
    print('----------------------------------------------------------------------------')
    print(preprocessing(sample))

Bom Dia Tenho dois processos o qual ja voo , nao conseguimos concluir a 
manifestacao os dados foram informados ,porem na ultima te (resumo) ela nao 
conclui, permanece na mesma, sem efetuar a manifestacao, sendo que a mesma 
ja saiu do estoque, ja esta desembaracada e ja voou. Oa AWBS e DUES 
respectivos sao 00174790763 (DUE 17BR0000049343) e 00174790660 
(DUE17BR0000043760) , ambos na mesma situacao. DATA 13-11-2017 HORARIO: 10 
40 [Attachment portalunico_siscomex_gov_br_cctPRINT3431510576873800.pdf 
processed by e-mail listener.] [Attachment 
portalunico_siscomex_gov_br_cctPRINT7601510576875486.pdf processed by 
e-mail listener.]
----------------------------------------------------------------------------
bom dia doi process voo nao consegu conclu manifestaca dad inform por ult resum nao conclu permanec mesm efetu manifestaca send mesm saiu estoqu desembarac voou awb due respec sao due duebr ambos mesm situaca dat horari attachment portalunicosiscomexgovbr processed listen attachmen

In [31]:
# Aplicando o pre-processamento
print('Iniciando a preparação das colunas textuais...')
start_time = time.time()
df['descricao_demanda2'] = df['descricao_demanda'].apply(preprocessing)
print('Processou a coluna descricao_demanda2 em %.2f segundos' % (time.time() - start_time))
start_time = time.time()
df['solucao_usuario2'] = df['solucao_usuario'].apply(preprocessing)
print('Processou a coluna solucao_usuario2 em %.2f segundos' % (time.time() - start_time))
start_time = time.time()
df['solucao_script2'] = df['solucao_script'].apply(preprocessing)
print('Processou a coluna solucao_script2 em %.2f segundos' % (time.time() - start_time))
# Combinando a classificação e grupo
df['classificacao_grupo'] = df['classificacao'].astype(str) + ' - ' + df['grupo'].astype(str)
df.head()

Iniciando a preparação das colunas textuais...
Processou a coluna descricao_demanda2 em 4.30 segundos
Processou a coluna solucao_usuario2 em 3.97 segundos
Processou a coluna solucao_script2 em 4.45 segundos


,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script,descricao_demanda2,classificacao_grupo,solucao_usuario2,solucao_script2
0,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO,UNACDFPUCOMEX,"Bom Dia Tenho dois processos o qual ja voo , n...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP,bom dia doi process voo nao consegu conclu man...,REGRA DE NEGÓCIO - UNACDFPUCOMEX,prez usuari carg esta american airlin american...,
3,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO,UNACDFPUCOMEX,"Srs., Fizemos várias tentativas para obter inf...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/00003655,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend...",fiz var tent obt informaco relaca adesa port d...,REGRA DE NEGÓCIO - UNACDFPUCOMEX,prez usuari voc pod est tir duvid tiag barbosa...,sra usuarioasu solicitaca iosamentecentr serv rfb
5,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,ANÁLISE URC,UNACDFPUCOMEX,"Recepcionamos a DUE 17BR0000257710, porem nao ...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",,"Caro Usuário,\n\nFavor informar a chave da not...",#N/DISP,recepcion due por nao segu desembarac,ANÁLISE URC - UNACDFPUCOMEX,car usuari favor inform chav not fiscal recepc...,
9,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,ANÁLISE URC,UNACDFPUCOMEX,A cia aérea não consegue averbar a DUE 17BR000...,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/0000002832,"\nPrezado(a) Usuário(a),\n\nA DUE encontra-se ...","Sr(a). Usuário(a),É necessário, para análise d...",cia aere nao conseg averb due poi aparec pes i...,ANÁLISE URC - UNACDFPUCOMEX,prezado usuario due encontras averb atencios c...,sra usuarioa necessari analis situaca acess pa...
11,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO,UNACDFPUCOMEX,GOSTARIA DE DETALHES DO MOTIVO DA NÃO AVERBAÇÃ...,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2016SOL/00397182,"Prezado Usuário,\n\nO manifestador precisa ret...","Prezado(a) usuário(a),conforme informado atrav...",gost detalh motiv nao averbaca due vez tod inf...,REGRA DE NEGÓCIO - UNACDFPUCOMEX,prez usuari manifest precis retific algum cois...,prezado usuarioaconform inform atrav contat te...


In [28]:
df.describe()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script,descricao_demanda2,classificacao_grupo,solucao_usuario2,solucao_script2
count,2302,2302,2302,2302,2302,2302,2302,2302,2302,2302,2302,2302
unique,1,5,17,2263,7,62,840,51,2188,32,739,45
top,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,DÚVIDA OU INFORMAÇÕES,ATGSMG,Usuário contatou o 0800 de atendimento SISCOM...,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",,"\nSr(a) usuário(a),\n\nInformamos que no ende...",#N/DISP,usuari contat atend siscomex por encerr ligaca...,DÚVIDA OU INFORMAÇÕES - ATGSMG,sra usuario inform enderec introdu const tod i...,
freq,2302,1258,903,6,1258,456,146,485,18,899,160,485


In [32]:
FILENAME = 'dataframe.pickle'
df.to_pickle(FILENAME)
print('arquivo salvo com sucesso!')

arquivo salvo com sucesso!


FIM

In [33]:
# Aplicando o pre-processamento
print('Iniciando a preparação das colunas textuais...')
start_time = time.time()
df2['descricao_demanda2'] = df2['descricao_demanda'].apply(preprocessing)
print('Processou a coluna descricao_demanda2 em %.2f segundos' % (time.time() - start_time))
start_time = time.time()
df2['solucao_usuario2'] = df2['solucao_usuario'].apply(preprocessing)
print('Processou a coluna solucao_usuario2 em %.2f segundos' % (time.time() - start_time))
start_time = time.time()
df2['solucao_script2'] = df2['solucao_script'].apply(preprocessing)
print('Processou a coluna solucao_script2 em %.2f segundos' % (time.time() - start_time))
# Combinando a classificação e grupo
df2['classificacao_grupo'] = df2['classificacao'].astype(str) + ' - ' + df2['grupo'].astype(str)
df2.head()

Iniciando a preparação das colunas textuais...
Processou a coluna descricao_demanda2 em 9.63 segundos
Processou a coluna solucao_usuario2 em 5.85 segundos
Processou a coluna solucao_script2 em 3.28 segundos


,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script,descricao_demanda2,solucao_usuario2,solucao_script2,classificacao_grupo
10,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Bom dia, ao tentarmos retificar a Due no porta...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/0000002898,Favor solicitar a\nreexecução da funcionalidad...,"Sr(a). Usuário(a),Para análise da situação, é ...",bom dia tent retific due port receb err inform...,favor solic reexecuca funcional poi ambi dispo...,sra usuarioap analis situaca necessari acess p...,SITUAÇÃO NÃO PREVISTA - CAGSBRPUCOMEX
12,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Sem acesso a criação de DU-E, opção de Embarqu...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2016SOL/00397102,"Prezado usuario,segue a informação do nosso de...","Sr(a). usuário(a), conforme informado através...",acess criaca due opca embarqu antecip vinculac...,prez usuarioseg informaca desenvolv problem an...,sra usuario conform inform atrav contat telefo...,SITUAÇÃO NÃO PREVISTA - CAGSBRPUCOMEX
23,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Bom dia, Conforme contato telefônico no qual f...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/00003655,"Prezado Edson,\nBom dia!\nA versão mais nova d...","\nSr(a). usuário(a),Sua solicitação foi atend...",bom dia conform contat telefon ger protocol se...,prez edson bom dia versa nov due produca hoj d...,sra usuarioasu solicitaca iosamentecentr serv rfb,SITUAÇÃO NÃO PREVISTA - CAGSBRPUCOMEX
26,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,O sistema está passando por uma indisponibilid...,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/0000006065,"Prezado Rafael,\nBoa tarde!\nConforme contato,...",p {\nmargin-bottom: 0.21cm;\n}\na:link {\n}\n\...,sistem pass indisponibil momentane possi agu a...,prez rafael boa tard conform contat document c...,marginbottom alink sra usuarioasu solicitaca i...,SITUAÇÃO NÃO PREVISTA - CAGSBRPUCOMEX
27,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRNOVOEX,Erro ao consultar DUE,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2016SOL/00400731,\n\n,"Sr(a) Usuário(a), A fim de darmos continuidad...",err consult due,,sra usuario fim darm continu demand solicit ut...,SITUAÇÃO NÃO PREVISTA - CAGSBRNOVOEX


In [34]:
FILENAME2 = 'dataframe2.pickle'
df2.to_pickle(FILENAME2)
print('arquivo salvo com sucesso!')

arquivo salvo com sucesso!
